In [5]:
import pymapee as pymap
import geemap as emap
import ee
pymap.initialize_ee()
pymap.authenticate_ee()

# 1. Masking cloud-related pixels

In [6]:
# MODIS NDVI Terra dataset
ndvi=ee.ImageCollection("MODIS/061/MOD13A2")

In [7]:
# Mask cloud-related pixels and slect NDVI band
modis_mask=pymap.modis_cloud_mask(ndvi, 2, 5, "DetailedQA", 1).select("NDVI")

In [8]:
# Make monthly NDVI composite
monthly_composite=pymap.monthly_composite(modis_mask, mode="median").select("NDVI")

In [11]:
# Scaling NDVI data
composite_scale=pymap.scaling_data(monthly_composite, 0.0001)

In [13]:
Map=emap.Map()
vis_params = {'bands': ['NDVI'], 'palette': ['9e0142', ' d0384e', ' ee6445', ' fb9c59', ' fece7c', ' fff1a7', ' f4faad', ' d1ec9c', ' 98d6a4', ' 5db7a9', ' 3683bb', ' 5e4fa2'], 'min': 0.0, 'max': 1}
Map.addLayer(composite_scale.first(),vis_params, "Monthly composite")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

# 2. Calculating Vegetation anomaly and condition indices

In [14]:
# Vegetation anomaly index (VAI)
vai=pymap.VAI(composite_scale)

In [15]:
Map=emap.Map()
vis_params = {'bands': ['NDVI'], 'palette': ['9e0142', ' d0384e', ' ee6445', ' fb9c59', ' fece7c', ' fff1a7', ' f4faad', ' d1ec9c', ' 98d6a4', ' 5db7a9', ' 3683bb', ' 5e4fa2'], 'min': -1, 'max': 1}
Map.addLayer(vai.first(),vis_params, "Monthly composite")
Map

EEException: User memory limit exceeded.

In [ ]:
# Vegetation condition index (VCI)
vci=pymap.VCI(composite_scale)

In [ ]:
Map=emap.Map()
vis_params = {'bands': ['NDVI'], 'palette': ['9e0142', ' d0384e', ' ee6445', ' fb9c59', ' fece7c', ' fff1a7', ' f4faad', ' d1ec9c', ' 98d6a4', ' 5db7a9', ' 3683bb', ' 5e4fa2'], 'min': -1, 'max': 1}
Map.addLayer(vai.first(),vis_params, "Monthly composite")
Map